In [ ]:
import numpy as np
import xarray as xr
import os,errno
import sys
import datetime as dt 
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
%matplotlib inline

dir='/thorncroftlab_rit/ahenny/rain/DISSERTATION_SCRIPTS_RESULTS/'
dir1='/thorncroftlab_rit/ahenny/rain/tccip_daily/'

In [ ]:
#This script will define the 99th percentile based on available years, then use that percentile to calculate the extreme rainfall on each Mei-Yu Season day.  The daily extreme rainfall totals will be ordered and the top 20% (n=79) selected.  Dates of these days will be collected and saved.
for i in range(41):
    print(i)
    year=1979+i
    base=dt.datetime(year,5,15,6)#middle of first day (starts 18Z on 14th)
    dates_list_mys=[base+dt.timedelta(days=x) for x in range(47)]
    ds=xr.open_dataset(dir1+'rain.'+str(year)+'.daily.1km-grid-v2.nc')
    p=ds.rain
    dates=[]
    for j in range(len(ds.time)):
        dates.append(dt.datetime(year,1,1,6)+dt.timedelta(j))
    p['time']=dates#now have datetime objects
    p_mys=p.sel(time=dates_list_mys)

    if i==0:
        p_mys_all=p_mys
    else:
        p_mys_all=xr.concat([p_mys_all,p_mys],dim='time')

#now calculate point-wise 99% threshold
threshold_array=xr.zeros_like(p_mys_all[0,:,:])
for j in range(p_mys_all.lat.size):
    print(j)
    for k in range(p_mys_all.lon.size):
        time_series=p_mys_all[:,j,k].values.tolist()
        measurable=[x for x in time_series if x>=0.1]#0.1 mm = measurable
        if len(measurable)>0:#areas outside island will be NaN
            threshold=np.percentile(measurable,99,interpolation='linear')
            threshold_array[j,k]=threshold
        else:
            threshold_array[j,k]=np.nan

#now get extreme rainfall during Mei-Yu season
p_mys_extreme=p_mys_all.where(p_mys_all>=threshold_array,drop=False)
p_mys_extreme_sum=p_mys_extreme.sum(dim=('lat','lon'),skipna=True)
extreme_sum_list=p_mys_extreme_sum.values.tolist()

In [ ]:
nonzero_extreme=[x for x in extreme_sum_list if x>0]
print('FINAL')
threshold_large_scale=np.percentile(nonzero_extreme,80,interpolation='linear')
large_scale_extreme_days=p_mys_extreme_sum.where(p_mys_extreme_sum>=threshold_large_scale,drop=True)
extreme_dates=large_scale_extreme_days.time
print(large_scale_extreme_days)
print(extreme_dates)
print(p_mys_extreme)
large_scale_extreme_rain=p_mys_extreme.sel(time=extreme_dates)
extreme_rain_all=p_mys_all.sel(time=extreme_dates)

In [ ]:
dk=xr.Dataset()
dk['large_scale_extreme_rain']=(('time','lat','lon'),large_scale_extreme_rain)
dk['large_scale_extreme_rain_all']=(('time','lat','lon'),extreme_rain_all)
dk['threshold']=(('lat','lon'),threshold_array)
dk.coords['time']=extreme_dates
dk.coords['lat']=large_scale_extreme_rain.lat
dk.coords['lon']=large_scale_extreme_rain.lon
try:
    os.remove(dir+'ls_extreme_rain_taiwan_99_80.nc')
except OSError:
    pass
dk.to_netcdf(dir+'ls_extreme_rain_taiwan_99_80.nc')

In [ ]:
#@author: Michael Schramm on GitHub
#This function is derived from code originally posted by Sat Kumar Tomer
#(satkumartomer@gmail.com)
#See also: http://vsp.pnnl.gov/help/Vsample/Design_Trend_Mann_Kendall.htm
from scipy.stats import norm
import scipy.stats as st
def mk_test(x, alpha=0.05):
    n = len(x)

    # calculate S
    s = 0
    for k in range(n-1):
        for j in range(k+1, n):
            s += np.sign(x[j] - x[k])

    # calculate the unique data
    unique_x, tp = np.unique(x, return_counts=True)
    g = len(unique_x)

    # calculate the var(s)
    if n == g:  # there is no tie
        var_s = (n*(n-1)*(2*n+5))/18
    else:  # there are some ties in data
        var_s = (n*(n-1)*(2*n+5) - np.sum(tp*(tp-1)*(2*tp+5)))/18

    if s > 0:
        z = (s - 1)/np.sqrt(var_s)
    elif s < 0:
        z = (s + 1)/np.sqrt(var_s)
    else: # s == 0:
        z = 0

    # calculate the p_value
    p = 2*(1-norm.cdf(abs(z)))  # two tail test
    h = abs(z) > norm.ppf(1-alpha/2)

    if (z < 0) and h:
        trend = 'decreasing'
    elif (z > 0) and h:
        trend = 'increasing'
    else:
        trend = 'no trend'

    return trend, h, p, z

In [ ]:
#now do threshold testing of AR-dominant EP day precipitation with different EP day thresholds
ds5=xr.open_dataset(dir+'taiwan_ar_climo_mys.nc')
print(ds5)
ar=ds5.ar
tc_trop=ds5.tc_trop
tc_nontrop=ds5.tc_nontrop
labeled=ds5['labeled']

ar_select=ar.sel(latitude=np.arange(21.25,26,0.25),longitude=np.arange(119.5,122.5,0.25))
ar_select=ar_select.fillna(0)
ar_select=ar_select.max(dim=('latitude','longitude'),skipna=True)

yrs_neusa=np.arange(1979,2020,1)
ar_days_list=[]
dates_all_list=[]
for i in range(len(yrs_neusa)):
    print(i)
    date_start=dt.datetime(yrs_neusa[i],5,14,18)
    for j in range(47):
        date_range=[date_start+dt.timedelta(days=j)+dt.timedelta(hours=6*x) for x in range(4)]
        ar_day=ar_select.sel(time=date_range).max(dim='time').values.tolist()
        ar_days_list.append(ar_day)
        dates_all_list.append(date_range[0])
    
print(len(ar_days_list))
print(47*41)
zipped_ar=list(zip(dates_all_list,ar_days_list))

In [ ]:
#trend testing
import pandas as pd
ds=xr.open_dataset(dir+'ls_extreme_rain_taiwan_99_80.nc')
p=ds['large_scale_extreme_rain_all']#total rainfall on ER days
dates=ds.time.values
print(len(dates))


p_means=p.mean(dim=('lat','lon'),skipna=True).values.tolist()
years=[pd.to_datetime(x).year for x in dates]
zipped_ep=list(zip(dates,p_means))

ar_values_list=[]
for i in range(len(p_means)):
    date=pd.to_datetime(dates[i])+dt.timedelta(hours=-12)#switch to 18Z so matches up
    
    select_ar=[x for x in zipped_ar if pd.to_datetime(x[0])==pd.to_datetime(date)]
    print(select_ar)
    ar_values_list.append(select_ar[0][1])

zipped_all=list(zip(years,p_means,ar_values_list))
totals_list=[]
for i in range(len(yrs_neusa)):
    select_year=[x for x in zipped_all if x[0]==yrs_neusa[i] and x[2]==1]
    ep_year=[x[1] for x in select_year]
    sum_year=float(sum(ep_year))
    totals_list.append(sum_year)
print(totals_list)

In [ ]:
linreg=st.linregress(yrs_neusa,totals_list)
slope1=linreg[0]
intercept1=linreg[1]
p_value1=linreg[3]
mk1=mk_test(totals_list,alpha=0.05)
if mk1[0] in ['increasing','decreasing']:
    mk1_str='MK-test = significant'
else:
    mk1_str='MK-test = insignificant'
if intercept1<0:
    eq_str='y='+str(round(slope1,2))+'x-'+str(abs(round(intercept1,2)))+', p='+str(round(p_value1,4))+' ) '
else:
    eq_str='y='+str(round(slope1,2))+'x+'+str(abs(round(intercept1,2)))+', p='+str(round(p_value1,4))+' ) '


fig=plt.figure(figsize=(12,12))
ax=plt.subplot(1,1,1)
ax.bar(yrs_neusa,totals_list,color='None',edgecolor='k',linewidth=2.0)
ax.bar(yrs_neusa,totals_list,color='g',label='AR-dominant ('+eq_str+' | '+mk1_str)
ax.set_title('99$^{th}$ / 80$^{th}$ (n='+str(len(dates))+' days)',fontsize=32,pad=10)
ax.tick_params(labelsize=19)
ax.set_xlabel('Year',fontsize=20,labelpad=15)
ax.set_ylabel('Total precipitation (mm)',fontsize=20,labelpad=15)
ax.set_ylim(0,700)
ax.set_xlim(1978.5,2019.5)
plt.legend(loc='upper left',fontsize=17)

plt.show()

In [ ]:
fig.savefig(dir+'precip_sensitivity_testing_taiwan_1.png')